In [180]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import itertools

# Пункт 1

In [208]:
# Наша метрика
def simple_metric(text, decode_text):
    len_text = min(len(text), len(decode_text))
    count = 0
    for x, y in zip(text, decode_text):
        if x == y:
            count += 1
    return count / len_text

In [181]:
# Функция для получения частотного словаря
def get_frequency(path):
    res = ''
    try:
        with open(path, 'r',  encoding='utf-8') as f:
            for line in f:
                res += re.sub(r'[^\w\s]', '', line.strip().lower()).replace('\t', ' ')
    except OSError or FileNotFoundError:
        res = path
    frequency = Counter(res)
    len_corpus = len(res)
    dict_frequency = {}
    for token in frequency.keys():
        dict_frequency[token] = frequency[token] / len_corpus
    return dict_frequency

In [182]:
# Получим частотный словарь на основе тренировочных данных "Война и Мир"
train_frequency = get_frequency('WarAndPeace.txt')

In [304]:
# Простенький энкодер
def simply_encoder(text='', parsed=False):
    if not parsed:
        text = re.sub(r'[^\w\s]', '', text.strip().lower()).replace('\t', ' ')
    count = Counter(text)
    token_list = list(count.keys())
    token_list_2 = token_list[:]
    random.shuffle(token_list_2)
    encode_dict = {}
    for ind in range(len(token_list)):
        encode_dict[token_list[ind]] = token_list_2[ind]
    res_text = ''
    for token in text:
        res_text += encode_dict[token]
    return res_text


def simply_decoder(test_text, train_frequency):
    test_frequency = get_frequency(test_text)
    
    res_train = []
    for key, val in train_frequency.items():
        res_train.append([key, val])
    res_train.sort(key=lambda x: x[1], reverse=True)
    
    res_test = []
    for key, val in test_frequency.items():
        res_test.append([key, val])
    res_test.sort(key=lambda x: x[1], reverse=True)
    
    decode_dict = {}
    for x, y in zip(res_test, res_train):
        decode_dict[x[0]] = y[0]

    res_text= ''
    for token in test_text:
        res_text += decode_dict[token]
    return res_text

In [184]:
# Прочтем тестовый текст
test_text = ''
with open('test.txt', 'r',  encoding='utf-8') as f:
        for line in f:
            test_text += re.sub(r'[^\w\s]', '', line.strip().lower()).replace('\t', ' ')

In [185]:
test_text[:500]

'действие трагедии уильяма шекспира король лир начинается в британии в замке короля герой одноимённого произведения представлен человеком имеющим неограниченную власть осознавая своё превосходство над окружающими он был уверен что царская сила велика а влияние его как отца на дочерей вечно однако пришла пора подумать пожилому человеку о своей будущей старости и замужестве дочерей гонерильи реганы корделии супруги герцогов корнуола и олбенипрежде чем указать какая часть наследства достанется каждо'

In [195]:
# Зашифруем текст
encode_text = simply_encoder(test_text)
encode_text[:500]

'ющванфдщзнрслщюддз1джй0ёсзещкапдрсзкуружйзждрзшсьдшсщна0зфзмрднсшддзфзысёкщзкуруж0злщрувзуюшудёяшшулузпрудыфщющшд0зпрщюансфжщшзьщжуфщкуёздёщц дёзшщулрсшдьщшш1цзфжсанйзуауышсфс0зафуязпрщфуатуюанфузшсюзукр1бсц дёдзушзмхжз1фщрщшзьнузосракс0заджсзфщждксзсзфжд0шдщзщлузкскзуносзшсзюуьщрщвзфщьшузуюшскузпрдежсзпурсзпую1ёснйзпубджуё1зьщжуфщк1зузафущвзм1ю1 щвзансруандздзысё1бщанфщзюуьщрщвзлушщрджйдзрщлсшхзкурющжддза1пр1лдзлщроулуфзкурш1ужсздзужмщшдпрщбющзьщёз1ксыснйзкскс0зьсанйзшсажщюанфсзюуансшщна0зксбюу'

In [206]:
# Попробуем его восстановить
simply_decoder(encode_text, train_frequency)[:500]

'дайлтриа тсеьадии мивяпге жаклуисе косовя вис небинеатлп р чситении р зегка косовп ьасой одноигхнноьо усоизраданип усадлтерван бавораког игаenиг наоьсенибаннмe рвелтя олознереп лрох усаролюодлтро нед оксмшеenиги он чыв мрасан бто цеслкеп ливе равике е рвипниа аьо кек отце не добасай рабно однеко усижве уосе уодмгетя уошивогм баворакм о лроай чмдмnай лтесолти и зегмшалтра добасай ьонасивяи саьены косдавии лмусмьи ьасцоьор коснмове и овчаниусашда баг мкезетя кекеп белтя нелвадлтре долтенатлп кешдо'

In [209]:
simple_metric(test_text, simply_decoder(encode_text, train_frequency))

0.5087303936075762

### Мы расшифровали с долей правильных расшифрованных бука ~ 0.5087303936075762

# Пункт 2

In [222]:
def get_bigram_frequency(path):
    res = ''
    try:
        with open(path, 'r',  encoding='utf-8') as f:
            for line in f:
                res += re.sub(r'[^\w\s]', '', line.strip().lower()).replace('\t', ' ')
    except OSError or FileNotFoundError:
        res = path
    bigram_list = [res[i - 1:i + 1] for i in range(1, len(res))]
    frequency = Counter(bigram_list)
    len_corpus = len(bigram_list)
    dict_frequency = {}
    for token in frequency.keys():
        dict_frequency[token] = frequency[token] / len_corpus
    return dict_frequency

In [242]:
train_frequency = get_bigram_frequency('WarAndPeace.txt')

In [226]:
len(train_frequency)

1575

In [248]:
def bigram_decoder(test_text, train_frequency):
    test_frequency = get_bigram_frequency(test_text)
    
    res_train = []
    for key, val in train_frequency.items():
        res_train.append([key, val])
    res_train.sort(key=lambda x: x[1], reverse=True)
    
    res_test = []
    for key, val in test_frequency.items():
        res_test.append([key, val])
    res_test.sort(key=lambda x: x[1], reverse=True)
    
    decode_dict = {}
    for x, y in zip(res_test, res_train):
        decode_dict[x[0]] = y[0]

    res_text= ''
    for ind in range(1, len(test_text) + 1, 2):
        try:
            res_text += decode_dict[test_text[ind - 1: ind + 1]]
        except KeyError:
            res_text = res_text[:-1]
            res_text += decode_dict[test_text[ind - 2: ind]]
    return res_text

In [256]:
bigram_decoder(encode_text, train_frequency)[:500]

'ор pинегиенаших и охил cе мигиулна пголая еравнеиднетьте кзаво бови лик ie итонору  стоба лякрытатчт нй крбр гь еео м аж бсеь  рриу туы поейрьесалышщеот vь чевысет коа лунута у оод ай хокавуажин ннеьнивикньрьеси скзаич дв м десп нчиn ан врыни криыле е сеееов иве нанннрочиале елпрм рав зя номнетоо воафе еннао омпан я ен жазий рриу туон нкнятраenмуусра сныкасяпожеекуктиин иелпрм раол твоили м имак пгоорери erй  ши ши фчту  пгочелае и латровй смор рэт данк ко пки у рт коалт няажине ел соттьте пa ел'

In [257]:
simple_metric(test_text, bigram_decoder(encode_text, train_frequency))

0.10298905001479727

In [319]:
def bigram_decoder_2(test_text, train_frequency):
    test_frequency = get_bigram_frequency(test_text)
    
    res_train = []
    for key, val in train_frequency.items():
        res_train.append([key, val])
    res_train.sort(key=lambda x: x[1], reverse=True)
    
    res_test = []
    for key, val in test_frequency.items():
        res_test.append([key, val])
    res_test.sort(key=lambda x: x[1], reverse=True)
    
    decode_dict = {}
    for x, y in zip(res_test, res_train):
        decode_dict[x[0]] = y[0]

    res_text= ''
    for ind in range(1, len(test_text) + 1, 1):
        try:
            if ind - 1 > 0:
                if (test_frequency[test_text[ind - 1: ind + 1]] <
                    test_frequency[test_text[ind - 2: ind]]):
                    res_text = res_text[:-1]
                    res_text += decode_dict[test_text[ind - 2: ind]]
                else:
                    res_text += decode_dict[test_text[ind - 1: ind + 1]][1:]
            else:
                res_text += decode_dict[test_text[ind - 1: ind + 1]][1:]
        except KeyError:
            pass
    return res_text

In [287]:
bigram_decoder_2(encode_text, train_frequency)[:500]

'оет сбы ионаи гои дгиебсе мтумеьае тгнлая ервлнедрнетье  лзаво бови ликекми тгнла   сноааомл еснтл лно нолев рьоввойм г  таяь рприув тоапеейь саа тчтнаоврьачеы кни с аклуунтау  оодаойм дкауом сд ннсва  икньь ес аскзбос д сть  ро ниныман  огие в ерне е кеееов иелн аннарихе ееоелрсм а виц наомнеонойваши о оа оеомус о оенжогы онприувтуна  оодтрзаумустао  бно си и к ийеи с  оелрсм раолтсвоили мвеоак тгоорери осйи ши  статоли тгочеаие  аларьовйм нр ипра данк о  анан  р  ся нт я г сееоса  б тьев аннел'

In [288]:
simple_metric(test_text, bigram_decoder_2(encode_text, train_frequency))

0.11397276494967436

### Результат не очень и хуже чем с униграмами и кажется это связано с тем, что я декодировал биграмами без пересечений (т.е. заменял зашифрованный текст кусками размером в 2 символа с шагом 2, соответсвующим по частоте биграмам трейна. 
### 2ой подход с биграмами, где рассматриваю пересечение 2ух биграм по одной букве и оставлю право выдать букву (на пересечении) той биграме, у которой вероятность появления больше, дало прибавку в одну сотую. Для примера тут имеется ввиду случай АБ БС и как их объединить, чтоб получить АБС. В общем случае у них не обязательно эта центральная буква будет одинакова.
### Думаю, что если выбирать биграму на замену из соображений какая буква первая была, то результат сильно улучшится 

### Еще предположение, что все таки во время трейна, мы оставили сликом много левых символов. Поэтому предлагаю повторить 1, 2 пункт по второму разу только с символами исключительно русского алфавита и пробела. Это могло дать много лишних комбинаций которые все и попортили.

### Повторим первый пункт

In [295]:
def get_frequency(path):
    res = ''
    try:
        with open(path, 'r',  encoding='utf-8') as f:
            for line in f:
                res += ''.join(re.findall(r'[а-я ]', line.strip().lower())).replace('\t', ' ')
                
    except OSError or FileNotFoundError:
        res = path
    frequency = Counter(res)
    len_corpus = len(res)
    dict_frequency = {}
    for token in frequency.keys():
        dict_frequency[token] = frequency[token] / len_corpus
    return dict_frequency

In [296]:
train_frequency = get_frequency('WarAndPeace.txt')

In [298]:
len(train_frequency)

33

In [299]:
# Прочтем тестовый текст
test_text = ''
with open('test.txt', 'r',  encoding='utf-8') as f:
        for line in f:
            test_text += ''.join(re.findall(r'[а-я ]', line.strip().lower())).replace('\t', ' ')

In [303]:
len(get_frequency(test_text))

32

In [305]:
# Видно что по размеру уже совпадают!
encode_text = simply_encoder(test_text)
encode_text[:500]

'щхпнуыохйутсехщоойяо лчьсйвхднцотсйдзтз лй отйэсмоэсхунчйыйртоусэоойыйфсьдхйдзтз чйехтзпйзщэзоьээзезйцтзофыхщхэочйцтхщнусы хэймх зыхдзьйоьхжбоьйэхзетсэомхээяжйы снулйзнзфэсысчйнызйцтхызназщнуызйэсщйздтяисжбоьойзэйрк йяыхтхэймузйшстндсчйно сйых одсйсйы очэохйхезйдсдйзушсйэсйщзмхтхпйыхмэзйзщэсдзйцтов сйцзтсйцзщяьсулйцзио зьяймх зыхдяйзйнызхпйрящябхпйнустзнуойойфсьяихнуыхйщзмхтхпйезэхто лойтхесэкйдзтщх оойняцтяеойехтшзезыйдзтэяз сйойз рхэоцтхищхймхьйядсфсулйдсдсчймснулйэсн хщнуысйщзнусэхунчйдсищзпй'

In [306]:
simply_decoder(encode_text, train_frequency)[:500]

'дайлтриа тсеьадии мивяпге жаклуисе косовя вис небинеатлп р чситении р зегка косовп ьасой одноигнноьо усоизраданип усадлтерван бавораког игахэиг наоьсенибаннмх рвелтя олознереп лро усаролюодлтро нед оксмшехэиги он чыв мрасан бто цеслкеп ливе равике е рвипниа аьо кек отце не добасай рабно однеко усижве уосе уодмгетя уошивогм баворакм о лроай чмдмэай лтесолти и зегмшалтра добасай ьонасивяи саьены косдавии лмусмьи ьасцоьор коснмове и овчаниусашда баг мкезетя кекеп белтя нелвадлтре долтенатлп кешдой '

In [307]:
simple_metric(test_text, simply_decoder(encode_text, train_frequency))

0.5135699373695198

In [308]:
### Результат стал чуть лучше для Униграм!!! Проверим на биграмах

### Повторим 2 пункт

In [309]:
def get_bigram_frequency(path):
    res = ''
    try:
        with open(path, 'r',  encoding='utf-8') as f:
            for line in f:
                res += ''.join(re.findall(r'[а-я ]', line.strip().lower())).replace('\t', ' ')
    except OSError or FileNotFoundError:
        res = path
    bigram_list = [res[i - 1:i + 1] for i in range(1, len(res))]
    frequency = Counter(bigram_list)
    len_corpus = len(bigram_list)
    dict_frequency = {}
    for token in frequency.keys():
        dict_frequency[token] = frequency[token] / len_corpus
    return dict_frequency

In [310]:
train_frequency = get_bigram_frequency('WarAndPeace.txt')
len(train_frequency)

815

In [311]:
bigram_decoder(encode_text, train_frequency)[:500]

'лооиинтислстших и дьилвуе милкымст вь ося  джеалидалтьте кзам какои лак фр  топоруда иоба крляысоволо пос орлоко нл  г стаегакреазортовалькеьнваонессткореатры кот ся й бяал е нхо пл бой айщеин палньивикумаюльи скзачироор баксп ппаушри ницот кпрмуе е сеутко  ы  приннвопагое в ре бноормя помалтоо м зве енсто омбуаня ен жазсадепрлиту т пхоусноссшешано сный сянелеукуючтин  в ре бноолонм или  бимн  вь ло ди згл ахи шиачесли вь  аосе и остркол халодеадрорик ра вки удет рагот егщеине в  сертьте вщав но'

In [312]:
simple_metric(test_text, bigram_decoder(encode_text, train_frequency))

0.09871756635848494

In [320]:
bigram_decoder_2(encode_text, train_frequency)[:500]

'лет сбы  ости гоирогигаее мтумевте тьпося  деоалдралтье  лзам какои лакуке  тьпос да пооаомовльтовлпо пос оронконол  г ктагнаеепрлортоанлкнюньагонессткоена аы кот с айичуатау  одпол бдй йом сд пасва  икумюнль аскзылсрор инаде  панымри  огте ор дие е к утко   л  ринавихе ленв е  бо оийвпаомалополм шт о  т оеошеена оенжог р тепрлотута  оодтнзаашеатоо капо си и к сатт ср нв е  бноолн м или  ы ен  ть ло ди оилиахида иссола ть  асте  аосрнкол бно  реарори на  рири де  ся ат г г сеени  кеотьен ринв он'

In [321]:
simple_metric(test_text, bigram_decoder_2(encode_text, train_frequency))

0.10023866348448687

### C Биграмами стало чуть хуже.